In [153]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

№3

In [154]:
p1 = 10.0/34
p2 = 4.0/16 

n1 = 34.0
n2 = 14.0

P = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
print P

0.3024647324296389


In [155]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [156]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [157]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [158]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [159]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [160]:
z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / (n1 + n2)))

In [161]:
print "p-value: %f" % proportions_diff_z_test(z_stat, alternative = 'greater')

p-value: 0.252883


№4

In [162]:
data = pd.read_csv('banknotes.txt', sep="\t", header=None)
data.columns = data.T[0].tolist()
data = data.drop([0])
data.head()

,X1,X2,X3,X4,X5,X6,real
1,214.8,131,131.1,9,9.7,141,1
2,214.6,129.7,129.7,8.1,9.5,141.7,1
3,214.8,129.7,129.7,8.7,9.6,142.2,1
4,214.8,129.7,129.6,7.5,10.4,142,1
5,215,129.6,129.7,10.4,7.7,141.8,1


In [163]:
X = data.drop(columns = ['real'])
y = data.real

In [164]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics
train_data, test_data, train_labels, test_labels = train_test_split(X,
                                                                    y,
                                                                    test_size = 0.25,
                                                                    random_state = 1)

In [165]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state = 0)
log_reg.fit(train_data.drop(columns = ['X4', 'X5', 'X6']), train_labels)
pred_123 = log_reg.predict(test_data.drop(columns = ['X4', 'X5', 'X6']))
metrics.mean_absolute_error(test_labels, pred_123)

0.2

In [166]:
log_reg = LogisticRegression(random_state = 0)
log_reg.fit(train_data.drop(columns = ['X1', 'X2', 'X3']), train_labels)
pred_456 = log_reg.predict(test_data.drop(columns = ['X1', 'X2', 'X3']))
metrics.mean_absolute_error(test_labels, pred_456)

0.02

In [167]:
test_lables_array = np.array([float(i) for i in test_labels])
pred_123_float = np.array([float(i) for i in pred_123])
pred_456_float = np.array([float(i) for i in pred_456])

abs_err_123 = np.absolute(test_lables_array - pred_123_float)
abs_err_456 = np.absolute(test_lables_array - pred_456_float)

In [168]:
proportions_diff_z_test(proportions_diff_z_stat_ind(abs_err_123, abs_err_456))

0.004022237272055307

№5

In [169]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_rel(abs_err_123, abs_err_456)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


№6

In [170]:
n = 2e6
S =100
mu = 525

n1 = 100
mu1 = 541.4

z = mu1 - mu
norm_dist = scipy.stats.norm(0, (S)/np.sqrt(n1))
print(2 * norm_dist.cdf(-z))

0.1010051669482079


№7

In [171]:
n = 2e6
S =100
mu = 525

n1 = 100
mu1 = 541.5

z = mu1 - mu
norm_dist = scipy.stats.norm(0, (S)/np.sqrt(n1))
print(2 * norm_dist.cdf(-z))

0.0989429360672962


1: $p_0$ +

2:(0,1) и (1,0)(1,0), и объёме выборок $n$+

3: 0.3025
    
    wrong:0.3787, 0.2483

4: 3 +
    

5:0.0599 +

6: 0.1010

7:0.0989